**Задача**

Найти решение следующей задачи:

$u_t - a^2(u_{xx}+u_{yy})= f(t, x, y)$ 

c начальными условиями: $u(0, x, y) = \varphi(x,y)$

с граничными условиями: $u(t,G) = \psi(t)$, где $G$ - граница области.

In [54]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
from dataclasses import dataclass, field

**Параметры задачи**

In [56]:
@dataclass
class EqParams:
    Lx: int   # Размер по оси Х
    Ly: int   # Размер по оси Y
    nx: int   # Количество точек по оси X
    ny: int   # Количество точек по оси Y
    nt: int   # Количество точек по оси t
    Tmax: int # Время моделирования
    a: float  # Коэффициент температуропроводности
    h_x: float = field(init=False) # Шаг по оси X
    h_y: float = field(init=False) # Шаг по оси Y
    tau: float = field(init=False) # Шаг по времени

    def __post_init__(self):
        self.h_x = self.Lx / (self.nx - 1)   # Расчёт шага по оси Х
        self.h_y = self.Ly / (self.ny - 1)   # Расчёт шага по оси Y
        self.tau = self.Tmax / (self.nt - 1) # Расчёт шага по времени

In [57]:
params = EqParams(Lx=100, Ly=100, nx=101, ny=101, nt=501, Tmax=50, a=1.25) # Задание параметров задачи

**Задание начальных условий**

In [59]:
def initial_cond(u: np.ndarray, params: EqParams) -> None:
    mid_x = int(params.nx/2)
    mid_y = int(params.ny/2)
    u[0, mid_x-10:mid_x+11, mid_y-10:mid_y+11] = 300 # Нагрев центра области

**Задание граничных условий**

In [61]:
def boundary_cond(u: np.ndarray, params: EqParams) -> None:
    u[:,0,:] = 75       # Условие на верхней стенке               
    u[:,params.nx-1,:] = 100   # Условие на нижней стенке
    u[:,:,0] = 75       # Условие на левой стенке
    u[:,:,params.ny-1] = -100  # Условие на правой стенке

**Задание функции источника**

In [63]:
def f(t: float, x: float, y: float) -> float:
    return np.sin(3*x + 5*y)

**Решение дифференциального уравнения**

In [65]:
def eq_heat(params: EqParams) -> np.ndarray:
    # u_t - a^2(u_xx + u_yy) = 0
    
    # Начальные и граничные условия
    u = np.zeros((params.nt, params.nx, params.ny))
    initial_cond(u, params)
    boundary_cond(u, params)

    # Решение дифференциального уравнения 
    for n in range(params.nt-1):
        for i in range(1, params.nx-1):
            for j in range(1, params.ny-1):
                u[n+1,i,j] = params.a**2*params.tau*((u[n,i+1,j] - 2*u[n,i,j] + u[n,i-1,j])/(params.h_x**2) + \
                                    (u[n,i,j+1] - 2*u[n,i,j] + u[n,i,j-1])/(params.h_y**2)) + \
                                   u[n,i,j] + params.tau*f(n, i, j)

    return u

**Создание анимации**

In [67]:
def create_animation(u: np.ndarray, params: EqParams) -> None:
    
    if not os.path.exists('frames'):
        os.makedirs('frames')
    
    for n in range(params.nt):
        fig, ax = plt.subplots()
        im = ax.imshow(u[n], cmap='coolwarm', vmin=-100, vmax=300)
        fig.colorbar(im)
        plt.title(f'Time: {(n)*params.tau:.2f}')
        plt.savefig(f'frames/{n}.png')
        plt.close(fig)

    images = []
    for n in range(params.nt):
        filename = f'frames/{n}.png'
        images.append(Image.open(filename).convert('RGB'))

    gif_filename = 'heat_equation.gif'
    images[0].save(gif_filename, save_all=True, append_images=images[1:], duration=50, loop=0)

In [68]:
create_animation(eq_heat(params), params)